In [1]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory
import time

In [2]:
Tarefas = [i for i in range(11)]
Maquinas = [1, 2]
processamento = {(0, 1): 0, (0, 2): 0,
                 (1, 1): 64,(1, 2): 64/.7,
                 (2, 1): 53,(2, 2): 53/.7,
                 (3, 1): 63,(3, 2): 63/.7,
                 (4, 1): 99,(4, 2): 99/.7,
                 (5, 1): 189,(5, 2): 189/.7,
                 (6, 1): 200,(6, 2): 200/.7,
                 (7, 1): 93,(7, 2): 93/.7,
                 (8, 1): 50,(8, 2): 50/.7,
                 (9, 1): 150,(9, 2): 150/.7,
                 (10, 1): 47,(10, 2): 47/.7,}
entrega = {0: 0, 1: 100, 2: 70, 3: 150, 4: 601, 5: 118, 6: 99, 7: 202, 8: 330, 9: 47, 10: 433}
R = 10**5

In [3]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.T = Set(initialize=Tarefas)
model.M = Set(initialize=Maquinas)
T, M = model.T, model.M

# Parâmetros
model.p = Param(T * M, initialize=processamento, within=NonNegativeReals)
model.d = Param(T, initialize=entrega, within=NonNegativeReals)
p, d = model.p, model.d

# Variáveis de decisão
model.C = Var(T * M, within=NonNegativeReals)
model.t = Var(T, within=NonNegativeReals)
model.x = Var(T * T * M, within=Binary)
C, t, x = model.C, model.t, model.x

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = sum(t[i] for i in T))

# Restrições
# Único sucessor
model.r_sucessor = ConstraintList()
for i in T:
    if i > 0:
        model.r_sucessor.add(expr = sum(sum(x[i, j, k] for j in T if j != i) for k in M) == 1)
# Precedência
model.r_proced = ConstraintList()
for k in M:
    for h in T:
        if h > 0:
            model.r_proced.add(expr = sum(x[i, h, k] for i in T if i != h) - sum(x[h, j ,k] for j in T if j != h) == 0)
# Uma sequência por máquina
model.r_seq = ConstraintList()
for k in M:
    model.r_seq.add(expr = sum(x[0, j, k] for j in T) <= 1)
# Instante de término
model.r_term = ConstraintList()
for i in T:
    for j in T:
        for k in M:
            if i != j and j > 0:
                model.r_term.add(expr = C[j, k] >= C[i, k] + p[j, k] * x[i, j, k] + R * (x[i, j, k] - 1))
# Atraso
model.r_atraso = ConstraintList()
for i in T:
    if i > 0:
        model.r_atraso.add(expr = t[i] >= sum(C[i, k] for k in M) - d[i])

# model.pprint()

In [4]:
# Resolução
start = time.time()
solver = SolverFactory('cplex')
resultado = solver.solve(model, tee=False)

In [5]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in T:
        for j in T:
            if i != j:
                for k in M:
                    if value(x[i, j, k]) == 1:
                        print('A tarefa {} precede a tarefa {} na máquina {}'.format(i, j, k))
    print()
    print('Atrasos')
    for i in T:
        if i > 0:
            print('Atraso da tarefa {}: {:.2f} minutos'.format(i, value(t[i])))
    print()
    print('Conclusões')
    for i in T:
        for k in M:
            if value(C[i, k]) > 0:
                print('Conclusão da tarefa {} na máquina {}: {:.2f} minutos'.format(i, k, value(C[i, k])))
    print()
    print('A soma dos atrasos foi de {:.2f} minutos'.format(value(model.obj)))
    
else:
    print(resultado.solver.termination_condition)
end = time.time()
print()
print('O tempo de resolução foi de {:.2f} segundos'.format(end - start))

A tarefa 0 precede a tarefa 1 na máquina 1
A tarefa 0 precede a tarefa 2 na máquina 2
A tarefa 1 precede a tarefa 3 na máquina 1
A tarefa 2 precede a tarefa 7 na máquina 2
A tarefa 3 precede a tarefa 9 na máquina 1
A tarefa 4 precede a tarefa 0 na máquina 2
A tarefa 5 precede a tarefa 4 na máquina 2
A tarefa 6 precede a tarefa 0 na máquina 1
A tarefa 7 precede a tarefa 5 na máquina 2
A tarefa 8 precede a tarefa 10 na máquina 1
A tarefa 9 precede a tarefa 8 na máquina 1
A tarefa 10 precede a tarefa 6 na máquina 1

Atrasos
Atraso da tarefa 1: 0.00 minutos
Atraso da tarefa 2: 5.71 minutos
Atraso da tarefa 3: 0.00 minutos
Atraso da tarefa 4: 19.00 minutos
Atraso da tarefa 5: 360.57 minutos
Atraso da tarefa 6: 475.00 minutos
Atraso da tarefa 7: 6.57 minutos
Atraso da tarefa 8: 0.00 minutos
Atraso da tarefa 9: 230.00 minutos
Atraso da tarefa 10: 0.00 minutos

Conclusões
Conclusão da tarefa 1 na máquina 1: 64.00 minutos
Conclusão da tarefa 2 na máquina 2: 75.71 minutos
Conclusão da tarefa 3 n